# Linear VMM (Vessel Maneuvering Model)

# Purpose
Implementing according to:
Matusiak, Jerzy. Dynamics of a Rigid Ship. Aalto University, 2017. https://aaltodoc.aalto.fi:443/handle/123456789/24408.

# Methodology
Define the problem using SymPy

# Setup

In [1]:
%config Completer.use_jedi = False

In [2]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from src.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from src.data import mdl




Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 463 ('figure.dpi       : 100        ## figure dots per inch')


In [3]:
u, v, r, delta = dynamicsymbols('u v r delta')

In [4]:
df_parameters = pd.DataFrame()
dofs = ['X','Y','N']
coords = ['u','v','r',r'\delta']
states = ['','dot']

for dof in dofs:
    
    for coord in coords:
        for state in states:
            key = f'{dof}{coord}{state}'
            key = key.replace('\\','')
            
            if len(state) > 0:
                symbol_name = r'%s_{\%s{%s}}' % (dof,state,coord)
            else:
                symbol_name = r'%s_{%s%s}' % (dof,state,coord)
            
            s = pd.Series(name=key, dtype='object')
            s['symbol'] = sp.symbols(symbol_name)
            s['dof'] = dof
            s['coord'] = coord
            s['state'] = state
            df_parameters = df_parameters.append(s) 
df_parameters = df_parameters.transpose()

In [5]:
df_parameters

,Xu,Xudot,Xv,Xvdot,Xr,Xrdot,Xdelta,Xdeltadot,Yu,Yudot,Yv,Yvdot,Yr,Yrdot,Ydelta,Ydeltadot,Nu,Nudot,Nv,Nvdot,Nr,Nrdot,Ndelta,Ndeltadot
coord,u,u,v,v,r,r,\delta,\delta,u,u,v,v,r,r,\delta,\delta,u,u,v,v,r,r,\delta,\delta
dof,X,X,X,X,X,X,X,X,Y,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,N,N
state,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot
symbol,X_{u},X_{\dot{u}},X_{v},X_{\dot{v}},X_{r},X_{\dot{r}},X_{\delta},X_{\dot{\delta}},Y_{u},Y_{\dot{u}},Y_{v},Y_{\dot{v}},Y_{r},Y_{\dot{r}},Y_{\delta},Y_{\dot{\delta}},N_{u},N_{\dot{u}},N_{v},N_{\dot{v}},N_{r},N_{\dot{r}},N_{\delta},N_{\dot{\delta}}


In [6]:
p = df_parameters.loc['symbol']

In [7]:
p.Xdelta

X_{\delta}

In [8]:
m,x_G,U,I_z = sp.symbols('m x_G U I_z')
X_eq = sp.Eq(
    (p.Xudot-m)*u.diff() + p.Xu*u + p.Xvdot*v.diff() + p.Xv*v + p.Xrdot*r.diff() + p.Xr*r + p.Xdelta*delta,
    0
)

Math(vlatex(X_eq))

<IPython.core.display.Math object>

In [9]:
Y_eq = sp.Eq(
    p.Yudot*u.diff() + p.Yu*u + (p.Yvdot-m)*v.diff() + p.Yv*v + (p.Yrdot-m*x_G)*r.diff() + (p.Yr-m*U)*r + p.Ydelta*delta,
    0
)
Math(vlatex(Y_eq))

<IPython.core.display.Math object>

In [10]:
N_eq = sp.Eq(
    p.Nudot*u.diff() + p.Nu*u + (p.Nvdot-m*x_G)*v.diff() + p.Nv*v + (p.Nrdot-I_z)*r.diff() + (p.Nr-m*x_G*U)*r + p.Ndelta*delta,
    0
)
Math(vlatex(N_eq))

<IPython.core.display.Math object>

In [11]:
u1d_eq = sp.Eq(u.diff(), sp.solve(X_eq, u.diff())[0])
Math(vlatex(u1d_eq))

<IPython.core.display.Math object>

In [12]:
v1d_eq = sp.Eq(v.diff(), sp.solve(Y_eq, v.diff())[0])
Math(vlatex(v1d_eq))

<IPython.core.display.Math object>

In [13]:
r1d_eq = sp.Eq(r.diff(), sp.solve(N_eq, r.diff())[0])
Math(vlatex(r1d_eq))

<IPython.core.display.Math object>

In [14]:
subs = {value:key for key,value in p.items()}
u1d_lambda = lambdify(u1d_eq.subs(subs).rhs)
v1d_lambda = lambdify(v1d_eq.subs(subs).rhs)
r1d_lambda = lambdify(r1d_eq.subs(subs).rhs)

## Parameters according to (Brix, 1993)

In [24]:
π = sp.pi
T,L,CB,B = sp.symbols('T L CB B')

df_parameters.loc['brix','Yvdot'] = -π*(T / L)**2 * (1 + 0.16*CB*B/T - 5.1*(B / L)**2)
df_parameters.loc['brix','Yrdot'] = -π*(T / L)**2 * (0.67*B/L - 0.0033*(B/T)**2)
df_parameters.loc['brix','Nvdot'] = -π*(T / L)**2 * (1.1*B/L - 0.04*(B/T))
df_parameters.loc['brix','Nrdot'] = -π*(T / L)**2 * (1/12 + 0.017*CB*B/T - 0.33*(B/L))
df_parameters.loc['brix','Yv'] = -π*(T / L)**2 * (1 + 0.4*CB*B/T)
df_parameters.loc['brix','Yr'] = -π*(T / L)**2 * (-1/2 + 2.2*B/L - 0.08*(B/T))
df_parameters.loc['brix','Nv'] = -π*(T / L)**2 * (1/2 + 2.4*T/L)
df_parameters.loc['brix','Nr'] = -π*(T / L)**2 * (1/4 + 0.039*B/T -0.56*B/L)

In [25]:
df_parameters.loc['T'] = 10
df_parameters.loc['L'] = 200
df_parameters.loc['CB'] = 0.7
df_parameters.loc['B'] = 30


In [29]:
df_parameters.loc['brix']

Xu                                                         NaN
Xudot                                                      NaN
Xv                                                         NaN
Xvdot                                                      NaN
Xr                                                         NaN
Xrdot                                                      NaN
Xdelta                                                     NaN
Xdeltadot                                                  NaN
Yu                                                         NaN
Yudot                                                      NaN
Yv                              -pi*T**2*(0.4*B*CB/T + 1)/L**2
Yvdot         -pi*T**2*(-5.1*B**2/L**2 + 0.16*B*CB/T + 1)/L**2
Yr                   -pi*T**2*(-0.08*B/T + 2.2*B/L - 0.5)/L**2
Yrdot             -pi*T**2*(-0.0033*B**2/T**2 + 0.67*B/L)/L**2
Ydelta                                                     NaN
Ydeltadot                                              

In [27]:
df_parameters.loc['value','Xu'] = 0

In [28]:
df_parameters.loc['value'].fillna(0, inplace=True)
df_parameters

,Xu,Xudot,Xv,Xvdot,Xr,Xrdot,Xdelta,Xdeltadot,Yu,Yudot,Yv,Yvdot,Yr,Yrdot,Ydelta,Ydeltadot,Nu,Nudot,Nv,Nvdot,Nr,Nrdot,Ndelta,Ndeltadot
coord,u,u,v,v,r,r,\delta,\delta,u,u,v,v,r,r,\delta,\delta,u,u,v,v,r,r,\delta,\delta
dof,X,X,X,X,X,X,X,X,Y,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,N,N
state,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot,,dot
symbol,X_{u},X_{\dot{u}},X_{v},X_{\dot{v}},X_{r},X_{\dot{r}},X_{\delta},X_{\dot{\delta}},Y_{u},Y_{\dot{u}},Y_{v},Y_{\dot{v}},Y_{r},Y_{\dot{r}},Y_{\delta},Y_{\dot{\delta}},N_{u},N_{\dot{u}},N_{v},N_{\dot{v}},N_{r},N_{\dot{r}},N_{\delta},N_{\dot{\delta}}
brix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-pi*T**2*(0.4*B*CB/T + 1)/L**2,-pi*T**2*(-5.1*B**2/L**2 + 0.16*B*CB/T + 1)/L**2,-pi*T**2*(-0.08*B/T + 2.2*B/L - 0.5)/L**2,-pi*T**2*(-0.0033*B**2/T**2 + 0.67*B/L)/L**2,NaN,NaN,NaN,NaN,-pi*T**2*(0.5 + 2.4*T/L)/L**2,-pi*T**2*(-0.04*B/T + 1.1*B/L)/L**2,-pi*T**2*(0.039*B/T - 0.56*B/L + 0.25)/L**2,-pi*T**2*(0.017*B*CB/T - 0.33*B/L + 0.08333333...,NaN,NaN
T,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
L,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
CB,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7
B,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30
value,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
u1d_lambda

<function _lambdifygenerated(Xdelta, Xr, Xrdot, Xu, Xudot, Xv, Xvdot, delta, m, r, r1d, u, v, v1d)>

In [ ]:
def step(t, X, parameters):
    
    
    